In [1]:
import pandas as pd
import numpy as np
from utils import *
from model import *

dep = 'SalePrice'
PATH = "data/bulldozers/"
df_raw = pd.read_feather('tmp/bulldozers-raw')
keep_cols = list(np.load('tmp/keep_cols.npy'))

df_raw.loc[df_raw.YearMade<1950, 'YearMade'] = 1950
df_raw['age'] = df_raw.saleYear-df_raw.YearMade
df_raw = df_raw[keep_cols+['age', dep]].copy()
df_indep = df_raw.drop(dep,axis=1)

n_valid = 12000
n_trn = len(df_raw)-n_valid


cat_flds = [n for n in df_indep.columns if df_raw[n].nunique()<n_trn/50]
' '.join(cat_flds)

for o in ['saleElapsed', 'saleDayofyear', 'saleDay', 'age', 'YearMade']: cat_flds.remove(o)
[n for n in df_indep.drop(cat_flds,axis=1).columns if not is_numeric_dtype(df_raw[n])]


for n in cat_flds: df_raw[n] = df_raw[n].astype('category').cat.as_ordered()

cont_flds = [n for n in df_indep.columns if n not in cat_flds]

df_raw = df_raw[cat_flds+cont_flds+[dep]]
df, y, nas, mapper = preprocessing(df_raw, 'SalePrice', do_scale=True)

val_idx = list(range(n_trn, len(df)))

def rmse(x,y): return np.sqrt(((x-y)**2).mean())
from sklearn.metrics import r2_score

emb_c = {n: len(c.cat.categories)+1 for n,c in df_raw[cat_flds].items()}

emb_szs = [(c, min(50, (c+1)//2)) for _,c in emb_c.items()]
metrics=[rmse]

In [2]:
train_loader, val_loader = StructuredData(df, y, cat_flds, cont_flds, batch_size=64, val_index=val_idx).get_data()

In [3]:
y_range=(0,np.max(y)*1.2)
model = MultiInputNN(emb_szs, len(cont_flds), 0.05, 1, [500,250], [0.5,0.05], use_bn=True, y_range=y_range)

In [4]:
fit(model, train_loader, loss=nn.MSELoss(), epochs=3, save=True, val_loader=val_loader, 
                                                    metrics=[rmse, r2_score], cycle_len=2, print_period=100)

A Jupyter Widget

[6000/6080 (99%)]
 [1, 0.1321422726304981, 0.07278734843002603, [0.26495359288179937, 0.5001124347239735]]
[6000/6080 (99%)]
 [2, 0.06792800130328702, 0.06623551519438226, [0.2532275194790274, 0.5213060141889929]]
[5000/6080 (82%)]

Process Process-5:
Traceback (most recent call last):
  File "/Users/mattevanoff/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/mattevanoff/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mattevanoff/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/Users/mattevanoff/anaconda3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/Users/mattevanoff/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
  File "/Users/mattevanoff/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/Users/mattevanoff/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chu

KeyboardInterrupt: 

In [ ]:
# print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.data[0]))